## 2018-10-20 Regular Expression Matching 正则表达式匹配
- 给定一个字符串 (s) 和一个字符模式 (p)。实现支持 '.' 和 '*' 的正则表达式匹配。
```
'.' 匹配任意单个字符。
'*' 匹配零个或多个前面的元素。
s 可能为空，且只包含从 a-z 的小写字母。
p 可能为空，且只包含从 a-z 的小写字母，以及字符 . 和 *。
```

In [ ]:
class Solution(object):
    def isMatch(self, s, p):
        

In [1]:
class Solution(object):
    def isMatch(self, s, p):
        # The DP table and the string s and p use the same indexes i and j, but
        # table[i][j] means the match status between p[:i] and s[:j], i.e.
        # table[0][0] means the match status of two empty strings, and
        # table[1][1] means the match status of p[0] and s[0]. Therefore, when
        # refering to the i-th and the j-th characters of p and s for updating
        # table[i][j], we use p[i - 1] and s[j - 1].

        # Initialize the table with False. The first row is satisfied.
        table = [[False] * (len(s) + 1) for _ in range(len(p) + 1)]

        # Update the corner case of matching two empty strings.
        table[0][0] = True

        # Update the corner case of when s is an empty string but p is not.
        # Since each '*' can eliminate the charter before it, the table is
        # vertically updated by the one before previous. [test_symbol_0]
        #p='a*';s=''
        for i in range(2, len(p) + 1):
            table[i][0] = table[i - 2][0] and p[i - 1] == '*'

        for i in range(1, len(p) + 1):
            for j in range(1, len(s) + 1):
                if p[i - 1] != "*":
                    # Update the table by referring the diagonal element.
                    table[i][j] = table[i - 1][j - 1] and \
                                  (p[i - 1] == s[j - 1] or p[i - 1] == '.')
                else:
                    # Eliminations (referring to the vertical element)
                    # Either refer to the one before previous or the previous.
                    # I.e. * eliminate the previous or count the previous.
                    # [test_symbol_1]
                    #p='ab*';s='a'
                    table[i][j] = table[i - 2][j] or table[i - 1][j]

                    # Propagations (referring to the horizontal element)
                    # If p's previous one is equal to the current s, with
                    # helps of *, the status can be propagated from the left.
                    # [test_symbol_2]
                    #p='ab*';s='abb'
                    if p[i - 2] == s[j - 1] or p[i - 2] == '.':
                        table[i][j] |= table[i][j - 1]

        return table[-1][-1]

In [2]:
a=Solution()
s='aab'
p='c*a*b'
c=a.isMatch(s,p)
print(s,p)
print(c)

aab c*a*b
True


In [3]:
s='mississippi'
p='mis*is*p*.'
c=a.isMatch(s,p)
print(s,p)
print(c)

mississippi mis*is*p*.
False


附记：
- 动态规划实际应用，核心思想表述：


初始状态为`dp[0][0] = true`表示两个空串相匹配，其他全部初始为false。

根据思路一的分析，我们有类似的状态转移方程：

* 如果模式最后一个字符（即`p[j-1]`）为$*$，则有两条路:
    * (1)尝试匹配0次，即`p[j-2]`和`p[j-1]`都没用，所以如果`dp[i][j-2] == true`则`dp[i][j]`为true；
    * (2)尝试匹配1次，即在前一个模式字符`p[j-2]`与待匹配字符`s[i-1]`相匹配的情况下，如果`dp[i-1][j]=true`则`dp[i][j]`为true；
* 否则，即模式最后一个字符不为'*'，则只能检查这个字符是否与待匹配字符是否匹配了，若匹配，且`dp[i-1][j-1]=true`则`dp[i][j]`为true。

$T(i,j)$表示的是$(p[0],p[i-1])$与$(s[0],s[j-1])$是否相符，则后续T更新动态规划规则如下：
$$T(i,j)=\begin{cases}T(i-1,j-1)==True \  and \ (p[i-1]==s[j-1]\ or\  p[i-1]=='.') & p[i-1]\ne '*' \\
(T(i-2,j)==True\ or\  T(i-1,j)==True)\ or\ ((p[i-2]==s[j-1]\ or\ p[i-2]=='.')\ and\ T(i,j-1)==True) & p[i-1]=='*'\\
\end{cases}
$$